In [12]:
import numpy as np
import os 

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score 

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense 
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

## Define Constants

In [3]:
from config import LABEL_MAP, DATA_LEN, DATA_PATH, NUM_EXAMPLES, SEQUENCE_LENGTH, ACTIONS, TEST_SPLIT, NUM_EPOCHS

## Get Data from Data Folder

In [4]:
# loop throuh each action folder, collect the example data and the appropraite label (enusre data has been created)

examples, labels = [], []
for action in ACTIONS: 
    for sequence in range(NUM_EXAMPLES):
        
        # load in all numpy data for a given example 
        window = []
        for frame in range(SEQUENCE_LENGTH):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame)))
            window.append(res)
        
        # for current example, add all data for that sequence and appropraite label 
        examples.append(window)
        labels.append(LABEL_MAP[action])

In [5]:
# convert data into appropraite format
X, y = np.array(examples), to_categorical(labels).astype(int)

In [6]:
# Split our data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT) # 5% to test and 95% to train 

In [7]:
# web app that allows you to see training
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [8]:
%load_ext tensorboard
%tensorboard --logdir Logs

Reusing TensorBoard on port 6006 (pid 20540), started 0:11:10 ago. (Use '!kill 20540' to kill it.)

## Build the model

In [9]:
# build model architecture 
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape = (SEQUENCE_LENGTH, DATA_LEN))) # to stack LSTMs return_sequences needs to be True, the next layer will recieve the "history"
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(ACTIONS.shape[0], activation='softmax'))


In [10]:
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

## Train Model and Save it

In [11]:
model.fit(X_train, y_train, epochs=NUM_EPOCHS, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 5s 111ms/step - loss: 1.2364 - categorical_accuracy: 0.3412
Epoch 2/2000
3/3 [==============================] - 0s 110ms/step - loss: 2.1693 - categorical_accuracy: 0.3176
Epoch 3/2000
3/3 [==============================] - 0s 109ms/step - loss: 1.1400 - categorical_accuracy: 0.3176
Epoch 4/2000
3/3 [==============================] - 0s 111ms/step - loss: 1.1095 - categorical_accuracy: 0.3529
Epoch 5/2000
3/3 [==============================] - 0s 112ms/step - loss: 1.1038 - categorical_accuracy: 0.3765
Epoch 6/2000
3/3 [==============================] - 0s 130ms/step - loss: 1.0962 - categorical_accuracy: 0.3529
Epoch 7/2000
3/3 [==============================] - 0s 121ms/step - loss: 1.1151 - categorical_accuracy: 0.4000
Epoch 8/2000
3/3 [==============================] - 0s 126ms/step - loss: 1.0986 - categorical_accuracy: 0.4118
Epoch 9/2000
3/3 [==============================] - 0s 142ms/step - loss: 1.1089 - categorical_accuracy:

KeyboardInterrupt: 

In [ ]:
# Save the model, you can name it whatever you want
model.save('Models/gesture.keras')

## Evaluate Model

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

accuracy_score(ytrue, yhat)

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)